In [ ]:
import rasterio 
from rasterio.merge import merge
import rasterio.plot
from rasterio.windows import Window
from rasterio.plot import show
import glob
from osgeo import gdal
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tables
import tqdm
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA, IncrementalPCA
from tqdm import tqdm
import matplotlib.pyplot as plt

# from osgeo import gdal
project_id = 'moja-328209'
#!gcloud config set project {project_id}
#!gsutil ls
np.seterr(divide='ignore', invalid='ignore')

{'divide': 'ignore', 'over': 'warn', 'under': 'ignore', 'invalid': 'ignore'}

# Changing resolution of files to 250

In [ ]:
from rasterio.enums import Resampling
def resample_raster(input_path, output_path, targetPixelSizeX = 0.002, targetPixelSizeY = 0.002):
    raster = rasterio.open(input_path)
    t = raster.transform

    pixelSizeX = t[0]
    pixelSizeY =-t[4]
    print(pixelSizeX, pixelSizeY)
    scalingFactorX = (targetPixelSizeX / pixelSizeX)
    scalingFactorY = (targetPixelSizeY / pixelSizeY)

    sizeX = raster.shape[0]
    sizeY = raster.shape[1]

    targetSizeX = int(sizeX / scalingFactorX)
    targetSizeY = int(sizeY / scalingFactorY)
    print(targetSizeX, targetSizeY)

    scale = scalingFactorX
    assert (scalingFactorX == scalingFactorY)

    # rescale the metadata
    transform = rasterio.Affine(t.a * scale, t.b, t.c, t.d, t.e * scale, t.f)  # <== multiplication
    height = int(raster.height // scale)                              # <== division
    width = int(raster.width // scale)
    print(height, width)

    profile = raster.profile
    profile.update(transform=transform, driver='GTiff', height=height, width=width, crs=raster.crs)
    print(profile)

    data = raster.read( # Note changed order of indexes, arrays are band, row, col order not row, col, band
            out_shape=(int(raster.count), int(height), int(width)),
            resampling=Resampling.cubic)

    with rasterio.open(output_path,'w', **profile) as dst:
        dst.write(data)

    print("Completed rewriting, now checking values:")
    resampled = rasterio.open(output_path)
    print('Orig dims: {}, New dims: {}'.format(raster.shape, resampled.shape))

from rasterio.enums import Resampling

def resampleRasterforNonSquare(input_path, output_path, targetPixelSizeX = 0.002, targetPixelSizeY = 0.002):
    raster = rasterio.open(input_path)
    t = raster.transform

    pixelSizeX = t[0]
    pixelSizeY =-t[4]
    print(pixelSizeX, pixelSizeY)
    scalingFactorX = (targetPixelSizeX / pixelSizeX)
    scalingFactorY = (targetPixelSizeY / pixelSizeY)

    sizeX = raster.shape[0]
    sizeY = raster.shape[1]

    targetSizeX = int(sizeX / scalingFactorX)
    targetSizeY = int(sizeY / scalingFactorY)
    print(targetSizeX, targetSizeY)

    scale = scalingFactorX

    # assert (scalingFactorX == scalingFactorY)

    # rescale the metadata
    transform = rasterio.Affine(t.a * scalingFactorX, t.b, t.c, t.d, t.e * scalingFactorY, t.f)  # <== multiplication
    height = int(raster.height // scalingFactorX)                              # <== division
    width = int(raster.width // scalingFactorY)
    print(height, width)

    profile = raster.profile
    profile.update(transform=transform, driver='GTiff', height=height, width=width, crs=raster.crs)
    print(profile)

    data = raster.read( # Note changed order of indexes, arrays are band, row, col order not row, col, band
            out_shape=(int(raster.count), int(height), int(width)),
            resampling=Resampling.cubic)

    with rasterio.open(output_path,'w', **profile) as dst:
        dst.write(data)

    print("Completed rewriting, now checking values:")
    resampled = rasterio.open(output_path)
    print('Orig dims: {}, New dims: {}'.format(raster.shape, resampled.shape))

In [ ]:
raster = rasterio.open("/data/uganda/budongo/matt/2001_resampled.tif")
t = raster.transform

pixelSizeX = t[0]
pixelSizeY =-t[4]
print(pixelSizeX, pixelSizeY)

0.002 0.002


In [ ]:
raster = rasterio.open("/data/uganda/budongo/corrected/2011_resampled.tif")
t = raster.transform

pixelSizeX = t[0]
pixelSizeY =-t[4]
print(pixelSizeX, pixelSizeY)

0.002 0.002


In [ ]:
#changing resolution of files after year 2000 only as matt hansen is available for that only
landsat_tifs = glob.glob("/data/uganda/budongo/corrected/2*.tif")
print(landsat_tifs)
for file in landsat_tifs:
  output_path = file[:-4] + "_resampled.tif"
  print(output_path)
  resampleRasterforNonSquare(file, output_path)

['/data/uganda/budongo/corrected/2011.tif', '/data/uganda/budongo/corrected/2004_resampled.tif', '/data/uganda/budongo/corrected/2018.tif', '/data/uganda/budongo/corrected/2000_resampled.tif', '/data/uganda/budongo/corrected/2010_resampled.tif', '/data/uganda/budongo/corrected/2018_resampled.tif', '/data/uganda/budongo/corrected/2009.tif', '/data/uganda/budongo/corrected/2020.tif', '/data/uganda/budongo/corrected/2005_resampled.tif', '/data/uganda/budongo/corrected/2007.tif', '/data/uganda/budongo/corrected/2016_resampled.tif', '/data/uganda/budongo/corrected/2016.tif', '/data/uganda/budongo/corrected/2003_resampled.tif', '/data/uganda/budongo/corrected/2015_resampled.tif', '/data/uganda/budongo/corrected/2012.tif', '/data/uganda/budongo/corrected/2017_resampled.tif', '/data/uganda/budongo/corrected/2008_resampled.tif', '/data/uganda/budongo/corrected/2001.tif', '/data/uganda/budongo/corrected/2000.tif', '/data/uganda/budongo/corrected/2013_resampled.tif', '/data/uganda/budongo/correct

KeyboardInterrupt: ignored

In [ ]:
#changing resolution of files after year 2000 only as matt hansen is available for that only
landsat_tifs = glob.glob("/data/uganda/budongo/matt/*.tif")
print(landsat_tifs)
for file in landsat_tifs:
  output_path = file[:-4] + "_resampled.tif"
  print(output_path)
  resampleRasterforNonSquare(file, othreeutput_path)

# Checking for null values

In [ ]:
for year in range(2001,2021):
  src = rasterio.open('/data/uganda/budongo/corrected/{}_resampled.tif'.format(year))
  X = src.read()
  isnull=0
  for i in range(0, 11):
    isnull+=np.isnan(X[i,:,:]).sum()

  print(year,":",isnull,"\tShape:",X.shape)
  src.close()
  del X


In [ ]:
for year in range(2001,2021):
  src = rasterio.open('/data/uganda/budongo/matt/{}_resampled.tif'.format(year))
  X = src.read()
  isnull=0
  for i in range(0, 1):
    isnull+=np.isnan(X[i,:,:]).sum()

  print(year,":",isnull,"\tShape:",X.shape)
  src.close()
  del X


# Finding min and max of each band

In [ ]:
tmp_max=-np.ones(11)*np.inf
tmp_min=np.ones(11)*np.inf
for year in range(2001,2021):
  print(year)
  src = rasterio.open('/data/uganda/budongo/corrected/{}_resampled.tif'.format(year))
  X = src.read()

  
  for i in range(0, 11):
    if tmp_max[i]<X[i,:,:].max():
      tmp_max[i]=X[i,:,:].max()
    if tmp_min[i]>X[i,:,:].min():
      tmp_min[i]=X[i,:,:].min()

In [ ]:
print("Max: ",tmp_max)
print("Min: ",tmp_min)

# PCA

In [ ]:
scaler = StandardScaler()

def extract_pixels(path):
  src = rasterio.open(path)
  print(path.split('/')[-1][:4])
  X = src.read()
  print(X.shape)
  # X = X[0:11]
  num_channels = X.shape[0]
  #num_channels = num_channels - 4
  print(X.shape)
  q = X.swapaxes(0, 2)
  del X
  print(q.shape)
  p = q.reshape(-1, num_channels)
  print(p.shape)
  del q
  df = pd.DataFrame(p)
  del p
  print(df.shape)
  # df = pd.concat([df, pd.DataFrame(data = y.ravel())], axis=1)
  df.columns= ["band{:02d}".format(i) for i in range(1, 1+num_channels)]
  file_name = "dataset_11bands" + path + ".csv"
  # df.to_csv(file_name)
  
  # print("Completed csv creation")
  return df, file_name

file_list = glob.glob('/data/uganda/budongo/corrected/*_resampled.tif')
file_list.sort()
# filtered_file_list = [file_list[1], file_list[5], file_list[10], file_list[15], file_list[20], file_list[25], file_list[30], file_list[34]]
file_list=file_list[1:]
print(file_list)

# use values from above to get proper scaling. Use these values in normalisation everywhere
max=[14491.696289,13768.695312,14429.892578,9878.423828,8770.582031,3953.342285,5805.459473,570.569824,31.853094,176.859222,0.000000]
min=[-339.67214966, -307.63830566, -341.41687012, -261.37908936, -208.54742432, -231.09004211, -131.0864563, -10.85133934, -1.09090972, -5.76107216, 0.0]
#X_std = (X - X.min(axis=0)) / (X.max(axis=0) - X.min(axis=0))
#X_scaled = X_std * (max - min) + min

def normalise(df):
  for k in range(df.shape[1]):
    if (max[k] - min[k])!=0:
      df.iloc[:, k] = -1 + 2*((df.iloc[:, k] - min[k])/(max[k] - min[k]))
  return df

In [ ]:
#detour to check the values - max and min
max_df = pd.DataFrame()
min_df = pd.DataFrame()

for file in file_list:
  df, _ = extract_pixels(file)
  min_df = min_df.append(df.min(), ignore_index=True)  
  max_df = max_df.append(df.max(), ignore_index=True)

print(max_df.max())
print(min_df.max())three

In [ ]:
batch_size = 1000
pca = IncrementalPCA(n_components = 11, batch_size=batch_size, copy=False)
pca_assumed3 = IncrementalPCA(n_components = 3, batch_size=batch_size, copy=False)
pca_assumed4 = IncrementalPCA(n_components = 4, batch_size=batch_size, copy=False)

for file in file_list:
  df_, file_name = extract_pixels(file)
  counter = 0
  total = 0
  df_ = normalise(df_)
  #break
  for batch in tqdm(range(0, df_.shape[0]-batch_size, batch_size)):
    total+=1
    try:  
      pca.partial_fit(df_[batch:batch+batch_size])
      pca_assumed3.partial_fit(df_[batch:batch+batch_size])
      pca_assumed4.partial_fit(df_[batch:batch+batch_size])
    except:  counter+=1
  # pca.partial_fit(df_[df_.shape[0]-batch_size-1:df_.shape[0]])
  # pca_assumed3.partial_fit(df_[df_.shape[0]-batch_size-1:df_.shape[0]])
  print(total, counter)
  print("-------------------------------")
print("Completed the work")

In [ ]:
ev=pca.explained_variance_ratio_
plt.figure(figsize=(12, 6))
plt.plot(np.cumsum(ev))
plt.xlabel('Number of components')
plt.ylabel('Cumulative explained variance')
plt.show()

In [ ]:
print(np.cumsum(ev))
pca_assumed4.components_

In [ ]:
pca_assumed3.components_

# Creating h5

In [1]:
import numpy as np
def make_ts(spectral_data, start, end, look_back):
  X = []
  for i in range(start+look_back, end):
    X_i = []
    for t in range(look_back):
      X_i.append(spectral_data[i-start-look_back+t])
    X.append(np.array(X_i))
  return np.array(X)

root = '/data/uganda/budongo/'

def make_dataset_and_save(filename, left_x, right_x, top_y, bottom_y, overlap, picture_size, look_back):
  year_targets = {}
  for year in range(2001, 2021):
    print("Opening", year)
    year_targets[year] = rasterio.open(root + "matt/" + str(year) + "_resampled.tif")

  geo_topleft_x, geo_topleft_y, geo_bottomright_x, geo_bottomright_y = 0, 0, 0, 0
  with rasterio.open(root + "corrected/2000_resampled.tif") as src:
     geo_topleft_x, geo_topleft_y = rasterio.transform.xy(src.transform, top_y, left_x)
     geo_bottomright_x, geo_bottomright_y = rasterio.transform.xy(src.transform, bottom_y, right_x)

  with rasterio.open(root + 'matt/2000_resampled.tif') as src: #As the middle file
      target_topleft_py, target_topleft_px = src.index(geo_topleft_x, geo_topleft_y)
      target_bottomright_py, target_bottomright_px = src.index(geo_bottomright_x, geo_bottomright_y)
  
  targets_file = rasterio.open(root + 'matt/2000_resampled.tif')
  shortlisted_coords = []

  for i in range(0, right_x-int(picture_size) + 1, int(picture_size*overlap)): #Loops in spectral data coordinates
    for j in range(0, bottom_y-int(picture_size) + 1,  int(picture_size*overlap)):
      y = targets_file.read(window=Window(target_topleft_px+i, target_topleft_py+j, picture_size, picture_size))
      y = y/100
      if np.sum(y) > 0.0:
        shortlisted_coords.append([i, j])
        
  targets_file.close()

  print("Shortlisted", len(shortlisted_coords))
  year_files = {}
  for year in range(2001, 2021):
    print("Opening", year)
    year_files[year] = rasterio.open(root + "corrected/" + str(year) + "_resampled.tif")

  for coords in shortlisted_coords:
    offset_x = coords[0]
    offset_y = coords[1]

    print("Starting", offset_x, offset_y)

    spectral_data = []
    target_data = []

    for year in range(2001, 2021):
      print("Appending", year)
      spectral_data.append(year_files[year].read(window=Window(offset_x, offset_y, picture_size, picture_size)))

    for year in range(2001+look_back, 2021):
      print("Generating targets", year)
      target_data.append(year_targets[year].read(window=Window(target_topleft_px+offset_x, target_topleft_py+offset_y, picture_size, picture_size)))

    print("Generating time series")

    X_thispatch = make_ts(spectral_data, 2001, 2021, look_back)
    print(X_thispatch.shape)

    y_thispatch = np.array(target_data)
    print(y_thispatch.shape)

    f_X = tables.open_file(filename_X, mode='a')
    f_y = tables.open_file(filename_y, mode='a')

    print("Writing patches...")
    for i in range(X_thispatch.shape[0]):
      f_X.root.data.append(np.reshape(X_thispatch[i], (1, look_back, num_channels, picture_size, picture_size)))
      f_y.root.data.append(np.reshape(y_thispatch[i], (1, 1, picture_size, picture_size)))
    f_X.close()
    f_y.close()
    print("Patches written")

  f_X = tables.open_file(filename_X, mode='a')
  f_y = tables.open_file(filename_y, mode='a')

  print(f_X.root.data.shape)
  print(f_y.root.data.shape)
  f_X.close()
  f_y.close()

In [2]:
tables.file._open_files.close_all()
filename_X = '/data/uganda/budongo/h5_resampled/X.h5'
filename_y = '/data/uganda/budongo/h5_resampled/y.h5'
num_channels = 11
look_back = 5
picture_size = 32
f_X = tables.open_file(filename_X, mode='w')
f_y = tables.open_file(filename_y, mode='w')
atom_X = tables.Float64Atom()
array_X = f_X.create_earray(f_X.root, 'data', atom_X, (0, look_back, num_channels, picture_size, picture_size))
atom_y = tables.Int8Atom()
array_y = f_y.create_earray(f_y.root, 'data', atom_y, (0, 1, picture_size, picture_size))
f_X.close()
f_y.close()

filename = ""

make_dataset_and_save(filename, 0, 643, 0, 643, 0.9, picture_size, look_back)

NameError: ignored

In [ ]:
# Generating shardsenter
PICTURE_SIZE = 32
import random
import tables
suffix = ['_000', '_001', '_002', '_003', '_004', '_005', '_006', '_007', '_008', '_009', '_010', '_011', '_012', '_013', '_014']
X_file = tables.open_file('/data/uganda/budongo/h5_resampled/X.h5', mode='r')
y_file = tables.open_file('/data/uganda/budongo/h5_resampled/y.h5', mode='r')
size_of_shard = X_file.root.data.shape[0]//15
indices = list(range(X_file.root.data.shape[0]))
num_channels = 11
look_back = 5
picture_size = PICTURE_SIZE
random.shuffle(indices)
for i in range(15):

  filename_X = '/data/uganda/budongo/h5_resampled/X'+suffix[i]+'.h5'
  filename_y = '/data/uganda/budongo/h5_resampled/y'+suffix[i]+'.h5'
  
  f_X = tables.open_file(filename_X, mode='w')
  f_y = tables.open_file(filename_y, mode='w')
  atom_X = tables.Float64Atom()
  array_X = f_X.create_earray(f_X.root, 'data', atom_X, (0, look_back, num_channels, picture_size, picture_size))
  atom_y = tables.Int8Atom()
  array_y = f_y.create_earray(f_y.root, 'data', atom_y, (0, 1, picture_size, picture_size))
  f_X.close()
  f_y.close()
  f_X = tables.open_file(filename_X, mode='a')
  f_y = tables.open_file(filename_y, mode='a')
  for j in range(size_of_shard):
    print("Shifting ", indices[i*size_of_shard + j], " to shard ", suffix[i])
    f_X.root.data.append(X_file.root.data[indices[i*size_of_shard + j]:indices[i*size_of_shard + j]+1, :, :, :, :])
    f_y.root.data.append(y_file.root.data[indices[i*size_of_shard + j]:indices[i*size_of_shard + j]+1, :, :, :])
  f_X.close()
  f_y.close()

In [ ]:
X_file = tables.open_file('/data/uganda/budongo/h5_resampled/X.h5', mode='r')
X_file.root.data.shape[0]

# Applying PCA to shards

In [ ]:
import h5py
for year in range(0,10):
  f = h5py.File("/data/uganda/budongo/h5_resampled/X_00"+str(year)+".h5","r")
  tm=np.array([f.get("data")])
  print(year,": ",np.sum(np.isnan(tm)))
for year in range(10,15):
  f = h5py.File("/data/uganda/budongo/h5_resampled/X_0"+str(year)+".h5","r")
  tm=np.array([f.get("data")])
  print(year,": ",np.sum(np.isnan(tm)))

In [ ]:
# applying pca to shards

# to process large h5 file
max=[14491.696289,13768.695312,14429.892578,9878.423828,8770.582031,3953.342285,5805.459473,570.569824,31.853094,176.859222,0.000000]
min=[-339.67214966, -307.63830566, -341.41687012, -261.37908936, -208.54742432, -231.09004211, -131.0864563, -10.85133934, -1.09090972, -5.76107216, 0.0]

class Transformer():
  def __init__(self, batch_size):
    tables.file._open_files.close_all()
    self.batch_size = batch_size
    self.num_channels_target = 4
    self.num_channels_initial = 11
    self.look_back = 5
    self.picture_size = PICTURE_SIZE
    self.shard_id = 0
    self.suffix = ['_000', '_001', '_002', '_003', '_004', '_005', '_006', '_007', '_008', '_009', '_010', '_011', '_012', '_013', '_014'] 
    # splitting the entire h5 data into 15 cohorts for faster i/o and preventing memory error
    X = tables.open_file('/data/uganda/budongo/h5_resampled/X'+self.suffix[self.shard_id]+'.h5', mode='r')
    self.size_of_shard = X.root.data.shape[0]
    self.num_of_batches_per_shard = int(np.ceil(self.size_of_shard / self.batch_size))
    self.x = self.x_corrected = 0

  def __len__(self):
    return self.num_of_batches_per_shard*len(self.suffix)
  
  def __initialiseFile__(self, shard_id):
    self.x = tables.open_file('/data/uganda/budongo/h5_resampled/X'+self.suffix[shard_id]+'.h5', mode='r')
    self.x_corrected = tables.open_file('/data/uganda/budongo/h5_resampled/X'+self.suffix[shard_id]+'_afterPCA.h5', mode='w')
    atom_X = tables.Float64Atom()
    array_X = self.x_corrected.create_earray(self.x_corrected.root, 'data', atom_X, (0, self.look_back, self.num_channels_target, self.picture_size, self.picture_size))
    self.x_corrected.close()
    self.x_corrected = tables.open_file('/data/uganda/budongo/h5_resampled/X'+self.suffix[shard_id]+'_afterPCA.h5', mode='a')

  def __closeFiles__(self):
    tables.file._open_files.close_all()

  def __getInfo__(self):
    idx_points = np.arange(0, self.size_of_shard+self.batch_size, self.batch_size)
    for idx in tqdm.tqdm(range(len(idx_points)-1)):
      batch_x = self.x.root.data[idx_points[idx]:idx_points[idx]+self.batch_size]   
      for k in range(batch_x.shape[2]):
        print(k, batch_x[:, :, k, :, :].max(), batch_x[:, :, k, :, :].min())
      break
    return

  def __transformBatch__(self, batch_x):
    #print("Batch Shape just after input: ", batch_x.shape)

    for k in range(batch_x.shape[2]):
      if(max[k] - min[k]):
        batch_x[:, :, k, :, :] = -1 + 2*((batch_x[:, :, k, :, :] - min[k])/(max[k] - min[k]))        

    transform = np.array([[ 0.12202053,  0.11268309,  0.12064926,  0.07898376,  0.09571855,
         0.01412919,  0.72256208,  0.64772149, -0.00160209, -0.02480929,
         0.        ],
       [ 0.51591997,  0.47636652,  0.4973759 ,  0.29864546,  0.32135193,
        -0.01210259, -0.11674392, -0.22759065,  0.00152171, -0.03856327,
        -0.        ],
       [ 0.01118694,  0.00860393,  0.01722012, -0.00163983,  0.0398583 ,
         0.01920229,  0.09191145, -0.07759655,  0.16495952,  0.97768278,
         0.        ],
       [-0.26001624, -0.17529508, -0.17275336,  0.15004874,  0.60731648,
         0.15153034,  0.44470698, -0.49954316, -0.03090505, -0.09616293,
         0.        ]])
   
    transformed_batch = np.zeros((1,5,4,PICTURE_SIZE,PICTURE_SIZE))
    for xx in range(batch_x.shape[0]):
      b = batch_x[xx]
      temp_storage = np.zeros((5,4,PICTURE_SIZE,PICTURE_SIZE))
      for i in range(b.shape[0]):
        c = b[i]
        d = np.dot(c.swapaxes(0, 2).reshape(-1, 11), transform.T).reshape((-1, PICTURE_SIZE, 4)).swapaxes(0, 2)
        temp_storage[i] = d
      transformed_batch[xx] = temp_storage
    
    return transformed_batch

  def __transformationAndAppend__(self):
    idx_points = np.arange(0, self.size_of_shard+self.batch_size, self.batch_size)
    #print(idx_points)
    for idx in tqdm.tqdm(range(len(idx_points)-1)):
      #print("Working from: ", idx_points[idx], idx_points[idx]+self.batch_size)
      batch = self.x.root.data[idx_points[idx]:idx_points[idx]+self.batch_size]
      transformedBatch = self.__transformBatch__(batch)
      #print("Batch Shape: ", batch.shape, " | Transformed Batch Shape: ", transformedBatch.shape)
      self.x_corrected.root.data.append(transformedBatch)

In [ ]:
import tqdm
PICTURE_SIZE = 32
TransformerClass = Transformer(1)
for i in range(len(TransformerClass.suffix)):
  print("Starting file: ", i)
  TransformerClass.__initialiseFile__(i)
  TransformerClass.__transformationAndAppend__()
  TransformerClass.__closeFiles__()

In [ ]:
import random

all_shard_list = ['_000', '_001', '_002', '_003', '_004', '_005', '_006', '_007', '_008', '_009', '_010', '_011', '_012', '_013', '_014']
# sampled_shard_list = random.sample(all_shard_list, 5)

min_cover = []
max_cover = []

for shard in all_shard_list:
  print("Opening shard ", shard)
  shard_min_cover = []
  shard_max_cover = []
  file_y = tables.open_file('/data/uganda/budongo/h5_resampled/y'+shard +'.h5', mode='r')
  for i in range(file_y.root.data.shape[0]):
    data = np.asarray(file_y.root.data[i][0])
    data = data/100.0
    data = np.where(data<0, 0.0, data)
    shard_min_cover.append(data.min())
    shard_max_cover.append(data.max())
  
  print("Done")
  min_cover.append(shard_min_cover)
  max_cover.append(shard_max_cover)


In [ ]:
min_cover = np.asarray(min_cover)
max_cover = np.asarray(max_cover)
global_min = min_cover.min()
global_max = max_cover.max()
print("Global Mean = ", (global_max+global_min)/2)
mean_min = 0
mean_max = 0
for i in range(max_cover.shape[0]):

  mean_min += min_cover[i][:].mean()
  mean_max += max_cover[i][:].mean()

mean_min /= 5
mean_max /= 5

print((mean_max+mean_min)/2)

In [ ]:
import h5py
for year in range(0,10):
  f = h5py.File("/data/uganda/budongo/h5_resampled/X_00"+str(year)+"_afterPCA.h5","r")
  tm=np.array([f.get("data")])
  print(year,": ",np.sum(np.isnan(tm)))
for year in range(10,15):
  f = h5py.File("/data/uganda/budongo/h5_resampled/X_0"+str(year)+"_afterPCA.h5","r")
  tm=np.array([f.get("data")])
  print(year,": ",np.sum(np.isnan(tm)))

# Training

In [ ]:
import tensorflow as tf
from tensorflow import keras
from keras import layers
from keras import backend as K
from keras.callbacks import EarlyStopping
from keras.callbacks import CSVLogger
from keras.callbacks import ModelCheckpoint
from keras.callbacks import TensorBoard
from keras.models import load_model
from tensorflow.keras.utils import Sequence
tf.test.gpu_device_name()
import tensorflow as tf
from keras import Model

print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

In [ ]:
# to process large h5 files
# use values from above to get proper scaling. Use these values in normalisation everywhere

class DeforDataSequence(Sequence):
  
  def __init__(self, batch_size, mode, targets = 'regression'):
    self.batch_size = batch_size
    self.mode = mode
    self.shards = 0
    self.folder = 'h5_resampled'
    self.targets = targets
    if self.mode == 'train':
      self.suffix = ['_000', '_001', '_002', '_003', '_004', '_005', '_006', '_007', '_008', '_009', '_010'] # splitting the entire h5 data into 15 cohorts for faster i/o and preventing memory error
      #self.suffix = ['_000']
    elif self.mode == 'test':
      self.suffix = ['_011', '_012', '_013', '_014']
    elif self.mode == 'new_patch':
      self.suffix = ['_000', '_001', '_002', '_003', '_004', '_005', '_006', '_007', '_008', '_009', '_010', '_011', '_012', '_013', '_014']
      self.folder = 'h5_patch2'
    self.x = tables.open_file('/data/uganda/budongo/'+self.folder+'/X'+self.suffix[self.shards]+'_afterPCA.h5', mode='r')
    self.y = tables.open_file('/data/uganda/budongo/'+self.folder+'/y'+self.suffix[self.shards]+'.h5', mode='r')
    self.size_of_shard = self.x.root.data.shape[0]
    self.num_of_batches_per_shard = int(np.ceil(self.size_of_shard / self.batch_size))

  def __len__(self):
    return self.num_of_batches_per_shard*len(self.suffix)
  
  def __getBatchForVisualisationX__(self, idx, item_ = "None"):
     if item_ is "None":
       item_ = self.suffix[self.shards]
     self.x = tables.open_file('/data/uganda/budongo/'+self.folder+'/X'+item_+'_afterPCA.h5', mode='r')
     batch_x = self.x.root.data[(idx-self.shards*self.num_of_batches_per_shard)*self.batch_size : (idx-self.shards*self.num_of_batches_per_shard + 1)*self.batch_size]
     self.x.close()
     return batch_x

  def __getBatchForVisualisationY__(self, idx, item_ = "None"):
     if item_ is "None":
       item_ = self.suffix[self.shards]
     self.y = tables.open_file('/data/uganda/budongo/'+self.folder+'/y'+item_+'.h5', mode='r')
     batch_y = self.y.root.data[(idx-self.shards*self.num_of_batches_per_shard)*self.batch_size : (idx-self.shards*self.num_of_batches_per_shard + 1)*self.batch_size]
     self.y.close()
     return batch_y

  def __getitem__(self, idx):
    batch_x = self.x.root.data[(idx-self.shards*self.num_of_batches_per_shard)*self.batch_size : (idx-self.shards*self.num_of_batches_per_shard + 1)*self.batch_size]
    batch_y = self.y.root.data[(idx-self.shards*self.num_of_batches_per_shard)*self.batch_size : (idx-self.shards*self.num_of_batches_per_shard + 1)*self.batch_size]

    batch_y = batch_y/100.0
    batch_y = np.where(batch_y<0.0, 0.0, batch_y)
    if self.targets == 'binary':
      batch_y = np.where(batch_y > 0.28, 1, 0).astype(int) #experiment - converting binary
    


    if (idx-self.shards*self.num_of_batches_per_shard + 1)*self.batch_size >= self.size_of_shard:
      self.x.close()
      self.y.close()
      self.shards = (self.shards+1)%len(self.suffix)
      self.x = tables.open_file('/data/uganda/budongo/'+self.folder+'/X'+self.suffix[self.shards]+'_afterPCA.h5', mode='r')
      self.y = tables.open_file('/data/uganda/budongo/'+self.folder+'/y'+self.suffix[self.shards]+'.h5', mode='r')
    return batch_x, batch_y

In [ ]:
#@title Hyperparams
# Do NOT modify code
id = "1_250" #@param {type:"string"}
loss_type = 'mean_squared_error' #@param ["weighted_binary_crossentropy", "binary_crossentropy", "hinge", "mean_squared_error"]
deforested_loss_weight = 100.0 #@param {type:"number"}
optimizer = 'adam' #@param ["adam", "adagrad", "adadelta"]
learning_rate =  0.0005 #@param {type:"number"}
batch_size =  64#@param {type:"integer"}
epochs =  100 #@param {type:"integer"}

In [ ]:
from tensorflow.keras.utils import plot_model
PICTURE_SIZE = 32
init = tf.keras.initializers.GlorotNormal()
def create_model():
  return keras.Sequential(
      [ 
      tf.keras.Input(shape=(5, 4, PICTURE_SIZE, PICTURE_SIZE)),
      tf.keras.layers.ConvLSTM2D(filters = 20, kernel_size = (3,3), padding = 'same', kernel_initializer=init, activation='relu', return_sequences = True, data_format = 'channels_first'),
      tf.keras.layers.BatchNormalization(),
      tf.keras.layers.ConvLSTM2D(filters = 15, kernel_size = (3,3), padding = 'same', activation='relu', return_sequences = True, data_format = 'channels_first'),
      tf.keras.layers.BatchNormalization(),
      tf.keras.layers.ConvLSTM2D(filters = 12, kernel_size = (3,3), padding = 'same', activation='relu', return_sequences = True, data_format = 'channels_first'),
      tf.keras.layers.BatchNormalization(),
      tf.keras.layers.ConvLSTM2D(filters = 10, kernel_size = (3,3), padding = 'same', activation='relu', return_sequences = True, data_format = 'channels_first'),
      tf.keras.layers.BatchNormalization(),
      tf.keras.layers.ConvLSTM2D(filters = 8, kernel_size = (3,3), padding = 'same', activation='relu', return_sequences = True, data_format = 'channels_first'),
      tf.keras.layers.BatchNormalization(),
      tf.keras.layers.ConvLSTM2D(filters = 6, kernel_size = (3,3), padding = 'same', activation='relu', return_sequences = True, data_format = 'channels_first'),
      tf.keras.layers.BatchNormalization(),
      tf.keras.layers.ConvLSTM2D(filters = 4, kernel_size = (3,3), padding = 'same', activation='relu', return_sequences = True, data_format = 'channels_first'),
      tf.keras.layers.BatchNormalization(),
      tf.keras.layers.ConvLSTM2D(filters = 3, kernel_size = (3,3), padding = 'same', activation='relu', return_sequences = True, data_format = 'channels_first'),
      tf.keras.layers.BatchNormalization(),
      tf.keras.layers.ConvLSTM2D(filters = 2, kernel_size = (3,3), padding = 'same', activation='relu', return_sequences = True, data_format = 'channels_first'),
      tf.keras.layers.BatchNormalization(),
      tf.keras.layers.ConvLSTM2D(filters = 1, kernel_size = (3,3), padding = 'same', activation='sigmoid', data_format = 'channels_first'),
      # tf.keras.layers.BatchNormalization(),
      # tf.keras.layers.Permute((2,3,1)),
      # tf.keras.layers.Conv2D(filters = 1, kernel_size = (3,3), padding='same', activation='sigmoid', data_format= 'channels_last'),
      # tf.keras.layers.Permute((3,1,2)), 
      ]
  )
model = create_model()
# plot_model(model, show_shapes=True)
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv_lstm2d_10 (ConvLSTM2D)  (None, 5, 20, 32, 32)    17360     
                                                                 
 batch_normalization_9 (Batc  (None, 5, 20, 32, 32)    128       
 hNormalization)                                                 
                                                                 
 conv_lstm2d_11 (ConvLSTM2D)  (None, 5, 15, 32, 32)    18960     
                                                                 
 batch_normalization_10 (Bat  (None, 5, 15, 32, 32)    128       
 chNormalization)                                                
                                                                 
 conv_lstm2d_12 (ConvLSTM2D)  (None, 5, 12, 32, 32)    11712     
                                                                 
 batch_normalization_11 (Bat  (None, 5, 12, 32, 32)   

In [ ]:
def weighted_binary_cross_entropy(weights: dict, from_logits: bool = False):
    def weighted_cross_entropy_fn(y_true, y_pred):
        tf_y_true = tf.cast(y_true, dtype=y_pred.dtype)
        tf_y_pred = tf.cast(y_pred, dtype=y_pred.dtype)
        weights_v = tf.where(tf.equal(tf_y_true, 1), deforested_loss_weight, 1.0)    #weights_v = tf.where(tf.equal(tf_y_true, 1), weights[1], weights[0])
        ce = K.binary_crossentropy(tf_y_true, tf_y_pred, from_logits=from_logits)
        loss = K.mean(tf.multiply(ce, weights_v))
        return loss
    return weighted_cross_entropy_fn

METRICS = [
  keras.metrics.TruePositives(name='tp',thresholds=0.15),
  keras.metrics.FalsePositives(name='fp',thresholds=0.15),
  keras.metrics.TrueNegatives(name='tn',thresholds=0.15),
  keras.metrics.FalseNegatives(name='fn',thresholds=0.15), 
  keras.metrics.BinaryAccuracy(name='accuracy'),
  keras.metrics.Precision(name='precision',thresholds=0.15),
  keras.metrics.Recall(name='recall',thresholds=0.15),
  keras.metrics.AUC(name='auc'),
]

loss_arg = None
if loss_type == 'weighted_binary_crossentropy':
  loss_arg = weighted_binary_cross_entropy({})
elif loss_type == 'binary_crossentropy':
  loss_arg = tf.keras.losses.BinaryCrossentropy()
elif loss_type == 'hinge':
  loss_arg = tf.keras.losses.Hinge()
elif loss_type == 'mean_squared_error':
  loss_arg = tf.keras.losses.MeanSquaredError(reduction=tf.keras.losses.Reduction.SUM)
elif loss_type == 'mean_absolute_error':
  loss_arg = tf.keras.losses.MeanAbsoluteError()

optimizer_arg = None
if optimizer == 'adam':
  optimizer_arg = keras.optimizers.Adam(learning_rate=learning_rate)
elif optimizer == 'adagrad':
  optimizer_arg = keras.optimizers.Adagrad(learning_rate=learning_rate)
elif optimizer == 'adadelta':
  optimizer_arg = keras.optimizers.Adadelta(learning_rate=learning_rate)

model_path = '/data/uganda/budongo/model/' + id 

In [ ]:
import datetime as time
model = create_model()
model.compile(loss=loss_arg, optimizer=optimizer_arg, metrics=METRICS)
model_checkpoint_callback = ModelCheckpoint(filepath=model_path)
path='/data/uganda/budongo/log/model_'+time.datetime.now().strftime("%Y%m%d-%H%M%S")
csv_logger = CSVLogger(path+'.log', append=True, separator=';')
tensorboard = TensorBoard(log_dir=path,histogram_freq=100)

import sys
sys.stdout = open('/dev/stdout', 'w')

# csv_logger = CSVLogger(sys.stdout)
# tensorboard = TensorBoard(log_dir=sys.stdout)

In [ ]:
%load_ext tensorboard

In [ ]:
%tensorboard --logdir logs

Reusing TensorBoard on port 6006 (pid 14611), started 0:15:34 ago. (Use '!kill 14611' to kill it.)

In [ ]:
data_sequence = DeforDataSequence(64, 'train')
model.fit(x = data_sequence, epochs = epochs, verbose = 2, shuffle=False, callbacks=[model_checkpoint_callback, csv_logger, tensorboard])

print(type(data_sequence))

In [ ]:
%tensorboard --logdir logs

Reusing TensorBoard on port 6006 (pid 14611), started 3:18:20 ago. (Use '!kill 14611' to kill it.)

# Testing

In [ ]:
model.save(path)

In [ ]:
config = model.get_config()
config

{'name': 'sequential_2',
 'layers': [{'class_name': 'InputLayer',
   'config': {'batch_input_shape': (None, 5, 4, 32, 32),
    'dtype': 'float32',
    'sparse': False,
    'ragged': False,
    'name': 'input_3'}},
  {'class_name': 'ConvLSTM2D',
   'config': {'name': 'conv_lstm2d_20',
    'trainable': True,
    'dtype': 'float32',
    'return_sequences': True,
    'return_state': False,
    'go_backwards': False,
    'stateful': False,
    'unroll': False,
    'time_major': False,
    'filters': 20,
    'kernel_size': (3, 3),
    'strides': (1, 1),
    'padding': 'same',
    'data_format': 'channels_first',
    'dilation_rate': (1, 1),
    'activation': 'relu',
    'recurrent_activation': 'hard_sigmoid',
    'use_bias': True,
    'kernel_initializer': {'class_name': 'GlorotNormal',
     'config': {'seed': None}},
    'recurrent_initializer': {'class_name': 'Orthogonal',
     'config': {'gain': 1.0, 'seed': None}},
    'bias_initializer': {'class_name': 'Zeros', 'config': {}},
    'unit_

In [ ]:
# def weighted_cross_entropy_fn(y_true, y_pred):
#         tf_y_true = tf.cast(y_true, dtype=y_pred.dtype)
#         tf_y_pred = tf.cast(y_pred, dtype=y_pred.dtype)
#         weights_v = tf.where(tf.equal(tf_y_true, 1), deforested_loss_weight, 1.0)    #weights_v = tf.where(tf.equal(tf_y_true, 1), weights[1], weights[0])
#         ce = K.binary_crossentropy(tf_y_true, tf_y_pred, from_logits=from_logits)
#         loss = K.mean(tf.multiply(ce, weights_v))
#         return loss

custom_objects = {"weighted_cross_entropy_fn": weighted_binary_cross_entropy}
# path='/data/uganda/budongo/log/model_20220116-102819'

In [ ]:
# with keras.utils.custom_object_scope(custom_objects):
model = keras.models.load_model(path)
data_sequence = DeforDataSequence(64, 'test')
result = model.evaluate(x = data_sequence, verbose = 0)
result = {out: result[i] for i, out in enumerate(model.metrics_names)}
test_results = pd.DataFrame(result, index = [0])

test_results.to_csv(path+'.csv')
print(test_results)

# temporary

In [ ]:
import h5py
for year in range(0,10):
  f = h5py.File("/data/uganda/budongo/h5_resampled/y_00"+str(year)+".h5","r")
  tm=np.array([f.get("data")])
  print("Shard ",year," : ",np.average(tm),"       ",np.std(tm))
for year in range(10,15):
  f = h5py.File("/data/uganda/budongo/h5_resampled/y_0"+str(year)+".h5","r")
  tm=np.array([f.get("data")])
  print("Shard",year," : ",np.average(tm),"       ",np.std(tm))

Shard  0  :  30.838128712551654         12.443926376295606
Shard  1  :  30.411774438274794         12.474120473133732
Shard  2  :  30.49998991154442         12.383110661506468
Shard  3  :  30.62018780668905         12.82502543412519
Shard  4  :  30.18637412835744         12.521378656216635
Shard  5  :  30.534244253615704         12.90562642997629
Shard  6  :  30.276859504132233         12.46106786185024
Shard  7  :  30.594066777505166         12.61309668435964
Shard  8  :  30.510092490960744         12.564163151383001
Shard  9  :  30.248452430914256         12.570671967452943
Shard 10  :  30.175454384039256         12.434286109525665
Shard 11  :  30.800280862603305         12.394219826061502
Shard 12  :  30.070145031637395         12.463815817114805
Shard 13  :  31.04171374612603         12.67045387742786
Shard 14  :  30.558248724819215         12.595474295316162
